In [ ]:
import torch
import numpy as np
from PIL import Image
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet
from facenet_pytorch import MTCNN
from facenet_pytorch import InceptionResnetV1
import cv2
from torch.nn.functional import normalize
import pyodbc as odbc
import os
import time



# Hyperparameters 
local_image_path = 'E:'  # Where your photos are stored locally
db_prefix_path = '\\\\172.16.17.136\\PHOTO_ROOT' # Will be replaced with the database path
min_face_size = 30 # Minimum faces you are allowing for the process eg. 30 (30*30 pixels)
SERVER_NAME = '192.168.1.101' 
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
DATABASE_NAME = 'AI_Face_Recognition'
idle_sleeptime = 1200 # How many seconds do you want the while to sleep when there is no row found in the Queue data
processing_sleeptime = 5 # How many seconds do you want the while to sleep when one game's process ends and another game's starts
retry_count = 3 # How many number of tries are you allowing to each game when they face any error
timestamp = 1200 # If any game has status InProgress, after how many seconds do you want to give that game a try
studio_margin = 80 # How much more of the face do you want in the studio images eg. 80 (from each side it will take 80 more pixels)
game_margin = 100 # How much more of the face do you want in the studio images eg. 80 (from each side it will take 80 more pixels)



class FinetunedMTCNN(MTCNN):
#(self, image_size=160, margin=5, **kwargs): # use this after some time to improve the final results
    def __init__(self, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)
        # Create your custom, finetuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()
    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)
    

facenet = InceptionResnetV1(pretrained='vggface2').eval() 
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device='cuda:0' if torch.cuda.is_available() else 'cpu', min_face_size=min_face_size)

def find_euclidean_distance(src, dst):
    """provides Euclidean distance for Face Alignment process."""
    return np.linalg.norm(src - dst)

def alignment_procedure(img, left_eye, right_eye): 
    """function takes the cropped face and returns aligned photo."""   
    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye
    # Find the direction to rotate the image based on the eye coordinates
    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1  # Clockwise
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1  # Counter-clockwise
    # Calculate the length of the triangle edges
    a = find_euclidean_distance(np.array(left_eye), np.array(point_3rd))
    b = find_euclidean_distance(np.array(right_eye), np.array(point_3rd))
    c = find_euclidean_distance(np.array(left_eye), np.array(right_eye))
    # Apply cosine rule to find the angle
    if b != 0 and c != 0:  # Avoid division by zero
        cos_a = (b**2 + c**2 - a**2) / (2 * b * c)
        angle = np.arccos(cos_a)  # Angle in radians
        angle = np.degrees(angle)  # Convert to degrees
        # Adjust the angle based on the rotation direction
        if direction == -1:
            angle = 90 - angle
        # Rotate the image using PIL
        #img = Image.fromarray(img)
        img = img.rotate(direction * angle, resample=Image.BICUBIC)
        img = np.array(img)  # Convert back to numpy array
        # aligned_img_pil = Image.fromarray(img)  # Convert back to PIL Image for saving
        # aligned_img_pil.show()
    return img

def detect_align_embed_faces(path, mtcnn_model, margin=0):
    """function takes a photo and provides embedding for the same."""
    global error_occurred
    cropped_photos = []
    # Check if the file exists at the given path
    if not os.path.exists(path):
        error_occurred = True
        return cropped_photos 
    try:
        image = Image.open(path)
        boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
        global photo_embeddings
        photo_embeddings = []
        if boxes is not None:
            # Set a confidence threshold
            threshold = 0.96
            # Filter detected faces based on the confidence score
            filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
            # Process each filtered face
            for i in filtered_faces:
                box = boxes[i]  # Get the bounding box for the filtered face
                box = [int(b) for b in box]  # Ensure the box is in integer format
                # adding margin around the box
                # Apply margin to the bounding box
                x1 = max(0, box[0] - margin)  # Left
                y1 = max(0, box[1] - margin)  # Top
                x2 = min(image.width, box[2] + margin)  # Right
                y2 = min(image.height, box[3] + margin)  # Bottom
                # Crop the face from the image
                cropped_face = image.crop((x1, y1, x2, y2))
                if cropped_face is not None: 
                    # Get the landmarks (left and right eyes) for the current face
                    left_eye = landmarks[i][0]  
                    right_eye = landmarks[i][1] 
                    # Align the cropped face using the eye coordinates
                    aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                    cropped_photos.append(aligned_face)
            for photo in cropped_photos:
                face_image = np.array(photo) # Convert the PIL image to a NumPy array
                image = cv2.resize(face_image, (160, 160))  # Resize to 160x160 as required by FaceNet
                image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float() / 255.0  # Convert to tensor, normalize
                embedding = facenet(image)  # Get the embedding
                embedding = normalize(embedding, p=2, dim=1)  # L2 normalization of embeddings
                embedding = embedding.detach().numpy()
                if embedding is not None:
                    photo_embeddings.append(embedding)  # Store the index and embedding
        return photo_embeddings
    except Exception as e:
        # If there is an error opening or processing the image
        print(f"Error processing image at {path}: {e}")
        # Update game status to error
        cur.execute("""UPDATE AITournamentQueue SET Status = 'error' WHERE GameNumber = ?""", (queue_data["GameNumber"],))
        cnxn.commit()
    return cropped_photos  # Continue with the next image

def euclidean_distance(embedding1, embedding2):
    return np.linalg.norm(embedding1 - embedding2)

threshold = 0.8

# Connection to database code:
cnxn = odbc.connect('DRIVER='+DRIVER_NAME+'; \
                    SERVER='+SERVER_NAME+'; \
                    DATABASE = '+DATABASE_NAME+'; \
                    Uid=sa;Pwd=Masterly@123;')
cur = cnxn.cursor()
cur.execute("use AI_Face_Recognition")


error_occurred = False
global count
count = 206 #(total number of games which are in pending state.)

while True:
    queue_query = """SELECT TOP 1 * FROM AITournamentQueue WHERE (Status IN('pending', 'error') AND RetryCount < ?) 
                        OR (Status = 'InProgress' AND DATEDIFF(SECOND, ProcessStartOn, CURRENT_TIMESTAMP) > ?)"""
    cur.execute(queue_query, (retry_count, timestamp))
    row_count = len(cur.fetchall())
    if row_count == 0:
        time.sleep(idle_sleeptime)
    else:
        try:
            cur.execute(queue_query, (retry_count, timestamp))
            queue_columns = [column[0] for column in cur.description]
            queue_rows = cur.fetchall()
            # This will fetch the games which have status pending
            for queue_row in queue_rows:
                count -= 1
                queue_data = dict(zip(queue_columns, queue_row))
                cur.execute("""UPDATE AITournamentQueue 
                            SET ProcessStartOn = CURRENT_TIMESTAMP, 
                                Status = 'InProgress' WHERE GameNumber = ?""", (queue_data["GameNumber"],))
                cnxn.commit()
                print(f"Process begin for Game: {queue_data['GameNumber']}")
                # Below query will return the studio photos for the particular game
                roster_query = """
                    SELECT * FROM CDPMediaCapture.dbo.Constellation 
                    WHERE PhotoType = 'Studio' AND TournamentID = ? AND MediaType = '.jpg' AND TeamKeys IN (?, ?) AND PhotoUse = ''"""
                cur.execute(roster_query, (queue_data['TournamentID'], queue_data['TeamKey1'], queue_data['TeamKey2']))
                roster_columns = [column[0] for column in cur.description]
                roster_rows = cur.fetchall()
                # for each studio photos, we will find embedding and adding it to the photoembedding table
                for roster_row in roster_rows:
                    roster_data = dict(zip(roster_columns, roster_row))
                    imageStoredPath = roster_data['KioskHiresFile'].replace(db_prefix_path, local_image_path)
                    photo_embeddings = detect_align_embed_faces(imageStoredPath, finetuned_mtcnn, margin=studio_margin)
                    # Checking if the embedding is already present or not for the specific studio photo
                    for embedding in photo_embeddings:
                        cur.execute("""
                            SELECT COUNT(*) FROM PlayerPhotoEmbedding
                            WHERE RosterID = ? AND TournamentID = ?
                        """, (roster_data["RosterID"], roster_data["TournamentID"]))
                        result = cur.fetchone()
                        if result[0] > 0:
                            pass
                        else:
                            cur.execute("""
                                INSERT INTO PlayerPhotoEmbedding (RosterID, TournamentID, SFaceEmbeddings, ImagePath, TeamKey, UpdatedOn, CreatedOn, ConstellationID)
                                VALUES (?, ?, ?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, ?)
                            """, (roster_data["RosterID"], roster_data["TournamentID"], embedding.tobytes(), imageStoredPath, roster_data["TeamKeys"], roster_data["ConstellationID"]))
                            cnxn.commit()
                print("Studio Photos Done")
                # Below query checks if there are already game photo embeddings exist in the GamePhotoEmbedding table, if then deletes them first then goes ahead again.
                cur.execute("DELETE FROM GamePhotoDetail WHERE GameNumber = ?", (queue_data["GameNumber"],))
                cur.execute("DELETE FROM GamePhotoEmbedding WHERE GameNumber = ?", (queue_data["GameNumber"],))
                # Below query will provide the game photos respective to the game number
                game_query = """SELECT * FROM CDPMediaCapture.dbo.Constellation WHERE MediaType = '.jpg' AND GameNumber = ?"""
                cur.execute(game_query, (queue_data["GameNumber"],))
                game_columns = [column[0] for column in cur.description]
                game_rows = cur.fetchall()
                # For each game photo, finding the number of faces and adding the details in the game photo detail table
                for game_row in game_rows:
                    game_data = dict(zip(game_columns, game_row))
                    game_image_path = game_data['KioskHiresFile'].replace(db_prefix_path, local_image_path)
                    photo_embeddings = detect_align_embed_faces(game_image_path, finetuned_mtcnn, margin=game_margin)
                    id = cur.execute("""
                        INSERT INTO GamePhotoDetail (NuFaceDetected, ImagePath, RosterIDs, GameNumber, TournamentID, TournamentWeek, TournamentWeekNumber, GameDay, GameTime, GameField, GameDayNumber, GameTimeNumber, GameType, U, B, R, ConstellationID, IsMatch, IsGroupPhoto, IsTagged, IsAttempted, UpdatedOn, CreatedOn) OUTPUT inserted.GamePhotoDetailID
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, NULL, NULL, NULL, NULL, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
                    """, (len(photo_embeddings), game_image_path, None, game_data["GameNumber"], game_data["TournamentID"], game_data["TournamentWeek"], game_data["TournamentWeekNumber"], game_data["GameDay"], game_data["GameTime"], game_data["GameField"], game_data["GameDayNumber"], game_data["GameTimeNumber"], game_data["GameType"], game_data["U"], game_data["B"], game_data["R"], game_data["ConstellationID"])).fetchval()
                    cnxn.commit()
                    # For each game photo, adding the embedding and gamephotodetailid into the gamephotoembedding table
                    for embedding in photo_embeddings:
                        cur.execute("""
                            INSERT INTO GamePhotoEmbedding (GamePhotoDetailID, GFaceEmbedding, UpdatedOn, CreatedOn, GameNumber)
                            VALUES (?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, ?)
                        """, (id, embedding.tobytes(), game_data['GameNumber']))
                        cnxn.commit()
                print("Game Photos Done.")
                cur.execute("""SELECT PlayerEmbeddingId, SFaceEmbeddings, RosterID FROM PlayerPhotoEmbedding WHERE TeamKey IN (?, ?)""", (queue_data['TeamKey1'], queue_data['TeamKey2']))
                player_columns = [column[0] for column in cur.description]
                player_embeddings = [dict(zip(player_columns, row)) for row in cur.fetchall()]
                # Process each studio embedding
                for player in player_embeddings:
                    s_embedding = np.frombuffer(player['SFaceEmbeddings'], dtype=np.float32).reshape(1, -1)
                    # Fetch game embeddings with the same GameNumber
                    cur.execute("""
                        SELECT GamePhotoEmbeddingId, GFaceEmbedding, GamePhotoDetailId 
                        FROM GamePhotoEmbedding 
                        WHERE GameNumber = ?
                    """, (game_data['GameNumber'],))
                    game_columns = [column[0] for column in cur.description]
                    game_embeddings = [dict(zip(game_columns, row)) for row in cur.fetchall()]
                    # Compare with relevant game embeddings
                    for game in game_embeddings:
                        g_embedding = np.frombuffer(game['GFaceEmbedding'], dtype=np.float32).reshape(1, -1)
                        distance = euclidean_distance(s_embedding, g_embedding)
                        if distance < threshold:
                            cur.execute("""
                                        UPDATE GamePhotoDetail
                                        SET MatchedFaces = MatchedFaces + 1,
                                            RosterIDs = COALESCE(RosterIDs + ' ', '') + ?
                                        WHERE GamePhotoDetailId = ?
                                    """, (player['RosterID'], game['GamePhotoDetailId']))
                            cnxn.commit()
                # this function will try to match the studio photos with the game photos
                print("Matching Task Done.")
                print(f"Games Remaining: {count}")
                print("---------------------------------------------------------------------------------------------------------------------------------------")  
                if error_occurred is False:
                    cur.execute("""UPDATE AITournamentQueue SET Status = 'completed', ProcessEndOn = CURRENT_TIMESTAMP WHERE GameNumber = ?""", (queue_data['GameNumber'],))
                    cnxn.commit()
                else:
                    cur.execute("""UPDATE AITournamentQueue SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP, RetryCount = RetryCount + 1 WHERE GameNumber = ?""", (queue_data['GameNumber'],))
                    cnxn.commit()
                    error_occurred = False
                time.sleep(processing_sleeptime)

        except Exception as e:
            # Log the error for the overall loop
            print(f"Error in the main loop: {e}")
            cur.execute("""UPDATE AITournamentQueue SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP, RetryCount = RetryCount + 1 WHERE GameNumber = ?""", (queue_data['GameNumber'],))
            cnxn.commit()
            break















In [ ]:
from tqdm import tqdm

# Assuming `game_rows` is a list of rows to process
total_images = len(game_rows)  # Total number of game images

# Wrap game_rows with tqdm for progress tracking
for game_row in tqdm(game_rows, desc="Processing Game Images", unit="image", total=total_images):
    game_data = dict(zip(game_columns, game_row))
    game_image_path = game_data['KioskHiresFile'].replace(db_prefix_path, local_image_path)
    
    # Detect, align, and get embeddings
    photo_embeddings = detect_align_embed_faces(game_image_path, finetuned_mtcnn, margin=game_margin)
    
    # Insert into GamePhotoDetail table
    id = cur.execute("""
        INSERT INTO GamePhotoDetail (NuFaceDetected, ImagePath, RosterIDs, GameNumber, TournamentID, TournamentWeek, TournamentWeekNumber, GameDay, GameTime, GameField, GameDayNumber, GameTimeNumber, GameType, U, B, R, ConstellationID, IsMatch, IsGroupPhoto, IsTagged, IsAttempted, UpdatedOn, CreatedOn) OUTPUT inserted.GamePhotoDetailID
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, NULL, NULL, NULL, NULL, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
    """, (len(photo_embeddings), game_image_path, None, game_data["GameNumber"], game_data["TournamentID"], game_data["TournamentWeek"], game_data["TournamentWeekNumber"], game_data["GameDay"], game_data["GameTime"], game_data["GameField"], game_data["GameDayNumber"], game_data["GameTimeNumber"], game_data["GameType"], game_data["U"], game_data["B"], game_data["R"], game_data["ConstellationID"])).fetchval()
    cnxn.commit()
    
    # Insert embeddings into GamePhotoEmbedding table
    for embedding in photo_embeddings:
        cur.execute("""
            INSERT INTO GamePhotoEmbedding (GamePhotoDetailID, GFaceEmbedding, UpdatedOn, CreatedOn, GameNumber)
            VALUES (?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, ?)
        """, (id, embedding.tobytes(), game_data['GameNumber']))
        cnxn.commit()


In [6]:
import os
import torch
import numpy as np
from PIL import Image, ImageDraw
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet
from facenet_pytorch import MTCNN

# 1. Define fine-tuned P-Net, R-Net, and O-Net for finetuning
class FinetunedMTCNN(MTCNN):
    def __init__(self, image_size=160, margin=15, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)

        # Create your custom, fine-tuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()

    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)

# Instantiate the fine-tuned MTCNN model
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device=device, min_face_size=60, thresholds=[0.6, 0.7, 0.7])

# 2. Function to perform face detection, apply NMS, and save cropped face images
def process_and_save_faces(image_folder, output_folder, mtcnn_model):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(image_folder, filename)
            image = Image.open(image_path)

            # Detect faces (with bounding boxes, probabilities, and landmarks)
            boxes, probs, landmarks = mtcnn_model.detect(image, landmarks=True)

            if boxes is not None:
                # Set a confidence threshold
                threshold = 0.96

                # Filter detected faces based on the confidence score
                filtered_faces = [i for i, prob in enumerate(probs) if prob > threshold]

                # Save cropped faces that pass the confidence threshold
                for i in filtered_faces:
                    box = boxes[i]  # Get the bounding box for the filtered face
                    cropped_face = image.crop(box)

                    # Save the cropped face image in the output folder
                    face_output_path = os.path.join(output_folder, f'face_{filename}_{i}.jpeg')
                    cropped_face.save(face_output_path)

# Specify your local directories
match_images_folder = r'D:/Images/Facial_reco_images/Facial_reco_images/photo'
output_folder_match = r'D:/Images/Facial_reco_images/Facial_reco_images/photo'
# studio_images_folder = 'path/to/local/studio_images_folder'
# output_folder_studio = 'path/to/local/output_folder_studio'

# Create the output directories if they don't exist
os.makedirs(output_folder_match, exist_ok=True)
# os.makedirs(output_folder_studio, exist_ok=True)

# Process both match and studio images
process_and_save_faces(match_images_folder, output_folder_match, finetuned_mtcnn)
# process_and_save_faces(studio_images_folder, output_folder_studio, finetuned_mtcnn)


In [ ]:
while True:
    try:
        queue_query = "SELECT TOP 10 * FROM AITournamentQueue WHERE Status = 'pending'"
        cur.execute(queue_query)
        queue_columns = [column[0] for column in cur.description]
        queue_rows = cur.fetchall()
        # This will fetch the games which have status pending
        for queue_row in queue_rows:
            count -= 1
            queue_data = dict(zip(queue_columns, queue_row))
            # Update status to 'In Progress'
            cur.execute("""UPDATE AITournamentQueue 
                           SET ProcessStartOn = CURRENT_TIMESTAMP, 
                               Status = 'In Progress' 
                           WHERE GameNumber = ?""", (queue_data["GameNumber"],))
            cnxn.commit()
            print(f"Process begin for Game: {queue_data['GameNumber']}")
            try:
                # Your main processing logic (roster_query, embedding generation, etc.)
                roster_query = """
                    SELECT * FROM CDPMediaCapture.dbo.Constellation 
                    WHERE PhotoType = 'Studio' AND TournamentID = ? AND MediaType = '.jpg' 
                    AND TeamKeys IN (?, ?) AND PhotoUse = ''"""
                cur.execute(roster_query, (queue_data['TournamentID'], queue_data['TeamKey1'], queue_data['TeamKey2']))
                # (Rest of the code for processing roster_rows, embeddings, etc.)
                # Update status to 'completed'
                cur.execute("""UPDATE AITournamentQueue 
                               SET Status = 'completed', ProcessEndOn = CURRENT_TIMESTAMP 
                               WHERE GameNumber = ?""", (queue_data['GameNumber'],))
                cnxn.commit()
            except Exception as e:
                # Log the error
                print(f"Error processing game {queue_data['GameNumber']}: {e}")
                
                # Update status to 'error' for this specific game
                cur.execute("""UPDATE AITournamentQueue 
                               SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP 
                               WHERE GameNumber = ?""", (queue_data['GameNumber'],))
                cnxn.commit()

            print("---------------------------------------------------------------------------------------------------------------------------------------")
    except Exception as e:
        # Log the error for the overall loop
        print(f"Error in the main loop: {e}")
        break
    finally:
        # Ensure that the status of all remaining games is set to 'error'
        cur.execute("""UPDATE AITournamentQueue 
                       SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP 
                       WHERE Status = 'In Progress'""")
        cnxn.commit()


In [ ]:
import os

def detect_align_embed_faces(path, mtcnn_model, margin=0):
    """Function takes a photo and provides embedding for the same."""
    cropped_photos = []
    
    # Check if the file exists at the given path
    if not os.path.exists(path):
        print(f"Error: Image not found at path {path}")
        # Assuming `queue_data["GameNumber"]` is available here, update the status to error
        cur.execute("""
            UPDATE AITournamentQueue 
            SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP 
            WHERE GameNumber = ?
        """, (queue_data["GameNumber"],))  # This assumes queue_data is available
        cnxn.commit()
        return cropped_photos  # Return empty to continue processing the next photo
    
    try:
        image = Image.open(path)  # Try to open the image
        # Continue with the rest of the image processing (cropping, embedding, etc.)
        # This part remains as it was before
        # Example (replace with your actual code):
        # cropped_photos = some_image_processing_function(image, mtcnn_model, margin)
        pass  # Replace with your actual image processing logic

    except Exception as e:
        # If there is an error opening or processing the image
        print(f"Error processing image at {path}: {e}")
        # Update game status to error
        cur.execute("""
            UPDATE AITournamentQueue 
            SET Status = 'error', ProcessEndOn = CURRENT_TIMESTAMP 
            WHERE GameNumber = ?
        """, (queue_data["GameNumber"],))
        cnxn.commit()
        
    return cropped_photos  # Continue with the next image



In [ ]:
def detect_align_embed_faces(image_path, output_folder, mtcnn_model, margin=0):
    """function takes a photo and provides embedding for the same."""
    cropped_photos = []
    image = Image.open(image_path)
    boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
    global photo_embeddings
    photo_embeddings = []
    if boxes is not None:
        threshold = 0.96
        filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
        for i in filtered_faces:
            box = boxes[i]  
            box = [int(b) for b in box]  
            x1 = max(0, box[0] - margin)  
            y1 = max(0, box[1] - margin)
            x2 = min(image.width, box[2] + margin) 
            y2 = min(image.height, box[3] + margin)  
            cropped_face = image.crop((x1, y1, x2, y2))
            if cropped_face is not None: 
                left_eye = landmarks[i][0]  
                right_eye = landmarks[i][1] 
                aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                face_output_path = os.path.join(output_folder, f'A_{i}.jpg')
                aligned_face.save(face_output_path)
                cropped_photos.append(aligned_face)
    return 


In [ ]:
import os
from PIL import Image

# Initialize a global counter for sequential naming
global_count = 1
def detect_align_embed_faces(image_path, output_folder, mtcnn_model, roster_id, margin=0):
    global global_count  # Use the global counter for naming
    cropped_photos = []
    image = Image.open(image_path)
    boxes, confidences, landmarks = mtcnn_model.detect(image, landmarks=True)
    global photo_embeddings
    photo_embeddings = []
    if boxes is not None:
        threshold = 0.96
        filtered_faces = [i for i, confidence in enumerate(confidences) if confidence > threshold]
        roster_folder = os.path.join(output_folder, str(roster_id))
        os.makedirs(roster_folder, exist_ok=True)
        for i in filtered_faces:
            box = boxes[i]
            box = [int(b) for b in box]
            x1 = max(0, box[0] - margin)
            y1 = max(0, box[1] - margin)
            x2 = min(image.width, box[2] + margin)
            y2 = min(image.height, box[3] + margin)
            cropped_face = image.crop((x1, y1, x2, y2))
            if cropped_face is not None:
                left_eye = landmarks[i][0]
                right_eye = landmarks[i][1]
                aligned_face = alignment_procedure(cropped_face, left_eye, right_eye)
                face_name = f'A{global_count}.jpg'
                face_output_path = os.path.join(roster_folder, face_name)
                aligned_face.save(face_output_path)
                global_count += 1    
    return cropped_photos


In [ ]:
import os
import numpy as np
from facenet_model import FaceNetModel  # Replace with your actual FaceNet implementation
from utils import load_image, save_image  # Utility functions for loading/saving images

# Load FaceNet model
facenet = FaceNetModel()

# Threshold for matching
threshold = 0.6  # Adjust based on your dataset

# Paths
studio_folder = "path_to_studio_photos"
game_folder = "path_to_game_photos"
output_folder = "path_to_store_results"

# Iterate over studio photos
for studio_filename in os.listdir(studio_folder):
    studio_path = os.path.join(studio_folder, studio_filename)
    studio_image = load_image(studio_path)
    
    # Get embedding for the studio photo
    studio_embedding = facenet.get_embedding(studio_image)
    
    # Create an output folder for the studio photo
    studio_output_folder = os.path.join(output_folder, os.path.splitext(studio_filename)[0])
    os.makedirs(studio_output_folder, exist_ok=True)
    
    # Store the studio photo
    save_image(studio_image, os.path.join(studio_output_folder, studio_filename))
    
    # Compare with game photos
    for game_filename in os.listdir(game_folder):
        game_path = os.path.join(game_folder, game_filename)
        game_image = load_image(game_path)
        
        # Get embedding for the game photo
        game_embedding = facenet.get_embedding(game_image)
        
        # Calculate Euclidean distance
        distance = np.linalg.norm(studio_embedding - game_embedding)
        
        # Check if the game photo matches
        if distance < threshold:
            # Store the game photo in the same folder as the studio photo
            save_image(game_image, os.path.join(studio_output_folder, game_filename))
